# 参数服务器
:label:`sec_parameterserver`

当我们从一个GPU迁移到多个GPU，然后再迁移到包含多个GPU的多个服务器时（可能全部分布在多个机架和网络交换机上），我们的分布式和并行训练算法需要变得更加复杂。细节很重要，因为不同的互连具有非常不同的带宽（例如，在适当的设置下，NVLink可以跨6条链路提供高达100 GB/s的带宽，PCIe 4.0（16通道）提供32 GB/s的带宽，而即使是高速100GbE以太网也只能提供10 GB/s）。同时，期望统计建模者成为网络和系统方面的专家是不合理的。

:cite:`Smola.Narayanamurthy.2010` 在分布式隐变量模型的背景下引入了参数服务器的核心思想。接着在 :cite:`Ahmed.Aly.Gonzalez.ea.2012` 中描述了Push和Pull语义，接着在 :cite:`Li.Andersen.Park.ea.2014` 中描述了系统和开源库。在下面，我们将描述提高效率所需的技术。

## 数据并行训练

让我们回顾一下分布式训练的数据并行训练方法。我们将在本节中使用它来排除所有其他内容，因为它在实践中的实现要简单得多。除了对图深度学习外，实际上没有任何场景推荐任何其他并行策略，因为GPU现在有大量的显存。 :numref:`fig_parameterserver` 描述了我们在 :numref:`sec_multi_gpu` 中实现的数据并行的变体。其中关键的一点是，在将更新的参数重新广播到所有GPU之前，在GPU 0上进行梯度聚合。

![左图：单GPU训练。右图：多GPU训练的一个变体：（1）计算损失和梯度，（2）所有梯度聚合在一个GPU上，（3）发生参数更新，并将参数重新分发给所有GPU。](../img/ps.svg)
:label:`fig_parameterserver`

回过头来看，在GPU 0上进行聚合似乎是拍脑瓜子的决定。毕竟，我们也可以在CPU上聚合。事实上，我们甚至可以决定在一个GPU上聚合一些参数，在另一个GPU上聚合其他一些参数。如果优化算法支持这一点，我们就没有理由不这样做。例如，如果我们有四个参数向量与相关的梯度$\mathbf{g}_1, \ldots, \mathbf{g}_4$，则我们可以针对每个$\mathbf{g}_i$将梯度聚集在一个GPU上($i = 1, \ldots, 4$)。

这种推理似乎是武断和轻率的。毕竟，数学自始至终都是一样的。但是，我们处理的是实际的物理硬件，其中不同的总线具有不同的带宽，如 :numref:`sec_hardware` 中所述。考虑一个真正的4路GPU服务器，如 :numref:`fig_bw_hierarchy` 中所述。如果它连接得特别好，它可能有一个100 GbE网卡。更典型的数字在1-10GbE范围内，有效带宽为100 MB/s到1 GB/s。由于CPU的PCIe通道太少，无法直接连接到所有GPU（例如，消费级Intel CPU有24个通道），因此我们需要[multiplexer](https://www.broadcom.com/products/pcie-switches-bridges/pcie-switches)。16x Gen3链路上CPU的带宽为16Gb/s。这也是每个GPU连接到交换机的速度。这意味着设备之间的通信更有效。

![一个4路GPU服务器](../img/bw-hierarchy.svg)
:label:`fig_bw_hierarchy`

为了便于讨论，让我们假设梯度是160MB。在这种情况下，将所有剩余3个GPU的梯度发送到第4个GPU需要30毫秒（每次传输需要10毫秒=160 MB/16 GB/s）。再加上30毫秒将权重向量传输回来，我们得到的结果是总共60毫秒。如果我们将所有数据发送到CPU，我们将有40毫秒的惩罚，因为4个GPU每个都需要将数据发送到CPU，总共产生80毫秒。最后，假设我们能够将梯度分为4部分，每个40 MB。现在我们可以在不同的GPU上同时聚合每个部分，因为PCIe交换机在所有链路之间提供全带宽操作。这需要7.5毫秒，而不是30毫秒。因此同步操作总共需要15毫秒。简而言之，根据我们同步参数的不同，同样的操作可能需要15ms到80ms不等的时间。 :numref:`fig_ps_distributed` 描述了交换参数的不同策略。

![参数同步策略](../img/ps-distributed.svg)
:label:`fig_ps_distributed`

请注意，我们还可以使用另一个工具来改进性能: 在深度网络中，计算从顶部到底部的所有梯度需要一些时间。我们可以开始同步一些参数的梯度，即使我们还在忙着为其他参数计算梯度。参见 :cite:`Sergeev.Del-Balso.2018` ，以了解在[Horovod](https://github.com/horovod/horovod)中如何做到这一点的详细信息。

## 环同步（Ring Synchronization）

当谈到现代深度学习硬件上的同步时，我们经常会遇到大量定制的网络连接。例如，AWS p3.16xlarge和NVIDIA DGX-2实例都使用到了 :numref:`fig_nvlink` 的连接结构。每个GPU通过PCIe链路连接到主机CPU，该链路最多只能以16 GB/s的速度运行。此外，每个GPU还具有6个NVLink连接，每个连接都能够双向传输300Gbit/s。这相当于每个方向每个链路约18 GB/s。简言之，聚合NVLink带宽明显高于PCIe带宽。问题是如何最有效地使用它。

![在8台V100 GPU服务器上连接NVLink（图片由英伟达提供）](../img/nvlink.svg)
:label:`fig_nvlink`

结果表明，最优的同步策略是将网络分解成两个环，并用它们直接同步数据 :cite:`Wang.Li.Liberty.ea.2018` 。 :numref:`fig_nvlink_twoloop` 说明了网络可以分解为一个具有双NVLink带宽的环（1-2-3-4-5-6-7-8-1）和一个具有常规带宽的环（1-4-6-3-5-8-2-7-1）。在这种情况下，设计一个高效的同步协议是非常重要的。

![将NVLink网络分解为两个环。](../img/nvlink-twoloop.svg)
:label:`fig_nvlink_twoloop`

考虑下面的想法：给定一个由$n$个计算节点（或GPU）组成的环，我们可以将梯度从第一个节点发送到第二个节点。在那里，它被添加到局部梯度并发送到第三个节点，依此类推。在$n-1$步之后，可以在最后访问的节点中找到聚合梯度。也就是说，聚合梯度的时间随节点数线性增长。但如果我们这样做，算法是相当低效的。毕竟，任何时候只有一个节点在通信。如果我们将梯度分为$n$个块，并从节点$i$开始同步块$i$，会怎么样？因为每个块的大小是$1/n$，所以总时间现在是$(n-1)/n \approx 1$。换句话说，当我们增大环的大小时，聚合梯度所花费的时间不会增加。这是一个相当惊人的结果。 :numref:`fig_ringsync` 说明了$n=4$个节点上的步骤顺序。

![跨4个节点的环同步。每个节点开始向其左邻居发送部分梯度，直到在其右邻居中找到聚合的梯度。](../img/ringsync.svg)
:label:`fig_ringsync`

如果我们使用相同的例子，跨8个V100GPU同步160MB，我们得到的结果大约是$2 \cdot 160 \mathrm{MB} / (3 \cdot 18 \mathrm{GB/s}) \approx 6 \mathrm{ms}$。这比使用PCIe总线要好，尽管我们现在使用的是8 GPU。请注意，在实践中，这些数字要更糟一些，因为深度学习框架通常无法将通信组合成大的突发传输。

请注意，有一种常见的误解，认为环同步与其他同步算法有根本的不同。唯一的区别是，与简单的树相比，同步路径稍微更精细一些。

## 多机训练

在多台机器上进行分布式训练还增加了一个挑战：我们需要与服务器通信，这些服务器只通过相对较低的带宽结构连接，在某些情况下，这种结构的速度可能会慢一个数量级。跨设备的同步很棘手。毕竟，运行训练代码的不同机器的速度会有细微的差别。因此，如果我们想使用同步分布式优化，我们需要同步这些机器。:numref:`fig_ps_multimachine`说明了分布式并行训练是如何发生的。

1. 在每台机器上读取一批（不同的）数据，在多个GPU之间划分并传输到GPU显存。在每个GPU上分别计算预测和梯度。
2. 来自所有本地GPU的梯度聚合在一个GPU上（或者它的一部分聚合在不同的GPU上）。
3. 梯度被发送到CPU。
4. CPU将梯度发送到中央参数服务器，该服务器聚合所有梯度。
5. 然后使用聚合梯度来更新参数，并将更新后的参数广播回各个CPU。
6. 信息被发送到一个（或多个）GPU。
7. 更新的参数分布在所有GPU上。

![多机多GPU分布式并行训练。](../img/ps-multimachine.svg)
:label:`fig_ps_multimachine`

这些操作似乎都相当简单。事实上，它们可以在一台机器内高效地执行。但是，一旦我们查看多台机器，我们就会发现中央参数服务器成为瓶颈。毕竟，每个服务器的带宽是有限的，因此对于$m$个工作节点来说，将所有梯度发送到服务器所需的时间是$\mathcal{O}(m)$。我们可以通过将服务器数量增加到$n$来突破这一障碍。此时，每个服务器只需要存储$\mathcal{O}(1/n)$个参数，因此更新和优化的总时间变为$\mathcal{O}(m/n)$。匹配这两个数字会产生恒定的伸缩性，而不管我们要处理多少工人。实际上，我们使用相同的机器作为工作节点和服务器。 :numref:`fig_ps_multips` 说明了设计（详见 :cite:`Li.Andersen.Park.ea.2014` ）。特别是，确保多台机器在没有不合理延迟的情况下工作是非常重要的。我们省略了关于阻塞的细节，下面只简单介绍一下同步和异步更新。

![上图：单参数服务器是一个瓶颈，因为它的带宽是有限的。下图：多参数服务器使用聚合带宽存储部分参数。](../img/ps-multips.svg)
:label:`fig_ps_multips`

## 键值存储

在实践中实现分布式多GPU培训所需的步骤绝非易事。这就是为什么使用一个公共抽象是值得的，即具有重新定义的更新语义的*键-值存储*的抽象。

在许多工作节点和许多GPU中，梯度$i$的计算可以定义为

$$\mathbf{g}_{i} = \sum_{k \in \text{workers}} \sum_{j \in \text{GPUs}} \mathbf{g}_{ijk},$$

其中$\mathbf{g}_{ijk}$是在工作节点$k$的GPU $j$上分划分的梯度$i$的一部分。这个运算的关键之处在于它是一个*交换归约*（commutative reduction），也就是说，它把许多向量变成一个向量，而运算的顺序并不重要。这对于我们的目的来说是非常好的，因为我们不需要对何时接收哪个梯度进行细粒度的控制。此外，请注意，此操作在不同的$i$之间是独立的。

这允许我们定义以下两个操作：*push*（累积梯度）和*pull*（检索聚合梯度）。因为我们有很多不同的梯度（毕竟，我们有很多层），所以我们需要用一个键$i$索引梯度。这种与键-值存储（ 如Dynamo :cite:`DeCandia.Hastorun.Jampani.ea.2007` 中引入的键-值存储）的相似性并非巧合。它们也满足许多类似的特性，特别是在多个服务器之间分配参数时。

键值存储的push-pull操作描述如下：

* **push（key，value）**将特定的梯度（value）从工作节点发送到公共存储。在那里，通过将其相加来聚合值。
* **pull（key，value）**从公共存储中检索聚合值，例如，在组合来自所有工作节点的梯度之后。

通过将同步的所有复杂性隐藏在一个简单的push和pull操作背后，我们可以将统计建模人员（他们希望能够用简单的术语表达优化）和系统工程师（他们需要处理分布式同步中固有的复杂性）的关注点解耦。

## 小结

* 同步需要高度适应特定的网络基础设施和服务器内的连接。这会对同步所需的时间产生重大影响。
* 对于p3和DGX-2服务器，环同步是最佳的。
* 当添加多个参数服务器以增加带宽时，分层同步策略可以很好地工作。

## 练习

1. 你能进一步提高环同步的性能吗？提示：你可以双向发送消息。
1. 是否可以允许异步通信（而计算仍在进行）？它如何影响性能？
1. 如果我们在长时间运行的计算过程中丢失了一台服务器，该怎么办？我们如何设计一种容错机制来避免完全重新启动计算？

[Discussions](https://discuss.d2l.ai/t/2807)
